#  </span> <span style="color:red"> LEZIONE 11  </span>
    
## ESERCIZIO 11.2
    
###  Funzione polinomiale $f(x)=4-3x-2x^2+3x^3$ con $x \in [-1,1]$.

Facendo diversi tentativi, a fissate caratteristiche, di *layer* e numero di neuroni per ogni *layer*, si nota come si riesce ad ottenere una predizione migliore con un numero più alto di *layer*, ciascuno con un numero di neuroni simile agli altri, piuttosto che con pochi *layer* e un alto numero di neuroni, nonostante nel secondo caso ci siano molti più parametri da ottimizzare e le perdite siano comunque basse. Per esempio (con funzione di attivazione *elu*, ottimizzatore *sgd* e funzione perdita *mse*) ho provato con:

- $2$ layer nascosti con rispettivamente $50$ e $30$ neuroni, per un totale di circa $1600$ parametri (Saved/Model_1).
- $4$ layer nascosti con un numero di neuroni simile, rispettivamente $15$, $25$, $10$ e $5$, per un totale di circa $750$ parametri (Saved/Model_2).

Tenendo invece fissi il numero di *layer* e di neuroni, in particolare usando il secondo caso precedente, ho provato a variare funzione di attivazione, con ottimizzatore *sgd* e funzione perdita *mse*. 

- Con *elu*, come nei casi precedenti, ho una buona precisione tranne nella zona $x \in [0.75, 1]$, in cui $f(x)$ raggiunge un minimo locale e torna ad aumentare.
- Con *relu* (impostazioni di default) invece nel range $x \in [0.75, 1]$ il modello si comporta molto bene, seguendo $f(x)$. Pecca di precisione, però, nella zona del massimo locale. (Saved/Model_4).
- Con *softplus* invece devo aumentare molto il numero $N_{epochs}$ perchè il rate di apprendimento è alto. Il grafico delle perdite è molto rumoroso, la precisione rispetto alla curva analitica è comunque minore rispetto ai casi precedenti. (Saved/Model_3).
- Anche con *tanh* la zona del minimo locale sembra ben fittata.

Date le considerazioni precedenti, ho provato ad usare entrambe le funzioni di attivazione *elu* (primo ed ultimo *layer*) e *relu* (*layer* centrali) nella speranza di poter combinare i "punti di forza" delle due. Il risultato, rispetto alla curva analitica $f(x)$ è sicuramente il migliore ottenuto, sia nella zona di massimo locale che in quella di minimo. (Saved/Model_5). Ho fatto lo stesso "mischiando" *relu* e *tanh* (Saved/Model_10), dovendo però aumentare il numero di iterazioni totali. I due modelli fittano grosso modo allo stesso modo.

Partendo dal *setting* *elu+relu*, ho provato a variare le funzioni di ottimizzazione. 
- Con *adadelta* (Saved/Model_6) e *adagrad* (Saved/Model_7) il risultato non sembra cambiare in modo sostanziale. Peggiora forse nella zona del minimo locale (che in ogni caso è quella che dà più problemi). 

Tenendo quindi le scelte di Saved/Model_5, ho provato a cambiare la funzione di perdita.
- Con *mae* perdo precisione nell'intorno del massimo locale, migliora nella zona minimo. (Saved/Model_8)
- Con altre funzioni ottengo risultato non consistenti.

Ho provato poi a guardare le predizioni fuori dal range di apprendimento del modello. Nelle zone fuori dall'intervallo $x \in [-1, 1]$ il modello non riesce a predire la funzione $f(x)$, come del resto ci si può aspettare. La rete ha imparato su determinati dati in un determinato range. Fuori da quello, a meno di essere fortunati (caso lineare dell'Esercizio_11.1 per esempio).


#### Genero i dati per l'allenamento e la validazione della rete neurale: lungo l'asse $y$ "sporco" con un rumore gaussiano a media nulla e scorrelato a punti diversi. Mostro la funzione bersaglio e il set di dati per la validazione.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize']=[8, 5]

# target parameters of f(x) = ax^3 + bx^2 + cx + d
a = 3
b = -2
c = -3
d = 4

# generate training inputs
np.random.seed(0)
x_train = np.random.uniform(-1, 1, 1000)
x_valid = np.random.uniform(-1, 1, 50)
x_valid.sort()
y_target = a*x_valid**3 + b*x_valid**2 + c*x_valid + d # ideal (target) function

sigma = 0.2 # noise standard deviation, for the moment it is absent
y_train = np.random.normal(a*x_train**3 + b*x_train**2 + c*x_train + d, sigma) 
y_valid = np.random.normal(a*x_valid**3 + b*x_valid**2 + c*x_valid + d, sigma)

# plot training and target dataset

plt.plot(x_valid, y_target, label='Target function')
plt.scatter(x_valid, y_valid, color='r', label='Validation data')
plt.legend(loc='upper right')
plt.grid(True); plt.show()

#### Genero la rete neurale scegliendo il numero e il tipo di *layer* con un rispettivo numero di neuroni (l'ultimo *layer* deve avere un solo neurone). Scelgo poi le funzioni di ottimizzazione, di perdita e la metrica. Inserisco anche una funzione di attivazione *elu*. Stampo il "riassunto" della rete che ho appena creato. 

In [ ]:
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras import backend as K
from tensorflow.keras.utils import get_custom_objects

In [ ]:
# compose the NN model
model = tf.keras.Sequential()
model.add(Dense(15, input_shape=(1,), activation='relu')) 
model.add(Dense(25, input_shape=(1,), activation='tanh')) 
model.add(Dense(10, input_shape=(1,), activation='relu')) 
model.add(Dense(5, input_shape=(1,), activation='tanh'))  
model.add(Dense(1, input_shape=(1,), activation='relu'))  #layer finale con 1 neurone

# compile the model choosing optimizer, loss and metrics objects
model.compile(optimizer='sgd', loss='mse', metrics=['mse'])

# get a summary of our composed model
model.summary()

#### Alleno la rete neurale: prende in entrata, per ogni iterazione, i dati di allenamento che ho generato prima e cerca i parametri che meglio descrivono la funzione dalla quale sono stati generati, minimizzando la funzione perdita.  

#### Stampo anche i risultati della validazione, cioè del test che faccio sul mio modello (con i dati di validazione generati in precedenza) per vedere quanto è buono il risultato. 

In [ ]:
# fit the model using training dataset
# over 10 epochs of 32 batch size each
# report training progress against validation data
history = model.fit(x=x_train, y=y_train, 
          batch_size=32, epochs=100,
          shuffle=True, # a good idea is to shuffle input before at each epoch
          validation_data=(x_valid, y_valid))

In [ ]:
# Stampa i pesi del modello, scommentare in caso
# model.get_weights()

#### Testo il modello finale che ho generato sui dati di validazione e sulla curva esatta per vedere quanto è buono

In [ ]:
# evaluate model
score = model.evaluate(x_valid, y_valid, batch_size=32, verbose=1)

# print performance
print()
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
# evaluate model with the exact curve
score = model.evaluate(x_valid, y_target, batch_size=32, verbose=1)

# print performance
print()
print('Test loss:', score[0])
print('Test accuracy:', score[1])

#### Riporto l'andamento delle perdite in relazione alle iterazioni per l'allenamento e il test sui dati di validazione

In [ ]:
# look into training history
# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='best')
plt.show()

#### Ora sono pronto per far predire dati sulla base del modello che ho creato e li confronto con la curva esatta

In [ ]:
x_predicted = np.random.uniform(-1, 1, 1000)
y_predicted = model.predict(x_predicted)
plt.scatter(x_predicted, y_predicted,color='r')
plt.plot(x_valid, y_target)
plt.grid(True)
plt.show()

### Salvare il Modello

In [ ]:
##### CELLA PER SALVARE IL MODELLO: CAMBIARE NOME, IN CASO ##########

save_model_path='Saved/Model_10'
model.save(filepath=save_model_path, include_optimizer=True)

### Caricare Modello salvato e stampare le caratteristiche

In [ ]:
# CARICARE UN MODELLO SALVATO #
# ricordarsi di compilare i primi 2 blocchi di questo jupyter #

from keras.models import load_model
from keras.utils import CustomObjectScope
from keras.initializers import glorot_uniform

with CustomObjectScope({'GlorotUniform': glorot_uniform()}):
        model_old = load_model('./Saved/Model_2')


# stampo le caratteristiche del modello salvato #
model_old.summary()

print()
print()
# evaluate model
print('Loss e accuracy sui dati di validazione')
print()
score = model_old.evaluate(x_valid, y_valid, batch_size=32, verbose=1)
# print performance
print()
print('Test loss:', score[0])
print('Test accuracy:', score[1])

print()
print()
# evaluate model with the exact curve
print('Loss e accuracy sulla funzione bersagio')
print()
score = model_old.evaluate(x_valid, y_target, batch_size=32, verbose=1)
print()
# print performance
print('Test loss:', score[0])
print('Test accuracy:', score[1])

### Predizioni

In [ ]:
x_predicted = np.random.uniform(-1, 1, 100)
y_predicted = model_old.predict(x_predicted)
plt.scatter(x_predicted, y_predicted,color='r')
plt.plot(x_valid, y_target)
plt.grid(True)
plt.show()